In [ ]:
import torchvision.models as models
import json
import argparse
import os

import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data.sampler import SequentialSampler
import numpy as np

from sklearn.cluster import DBSCAN, KMeans
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from tqdm import tqdm

In [ ]:
model_names = sorted(name for name in models.__dict__
                     if name.islower() and not name.startswith("__")
                     and callable(models.__dict__[name]))
print(model_names)

['alexnet', 'convnext_base', 'convnext_large', 'convnext_small', 'convnext_tiny', 'densenet121', 'densenet161', 'densenet169', 'densenet201', 'efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_b4', 'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7', 'efficientnet_v2_l', 'efficientnet_v2_m', 'efficientnet_v2_s', 'get_weight', 'googlenet', 'inception_v3', 'mnasnet0_5', 'mnasnet0_75', 'mnasnet1_0', 'mnasnet1_3', 'mobilenet_v2', 'mobilenet_v3_large', 'mobilenet_v3_small', 'regnet_x_16gf', 'regnet_x_1_6gf', 'regnet_x_32gf', 'regnet_x_3_2gf', 'regnet_x_400mf', 'regnet_x_800mf', 'regnet_x_8gf', 'regnet_y_128gf', 'regnet_y_16gf', 'regnet_y_1_6gf', 'regnet_y_32gf', 'regnet_y_3_2gf', 'regnet_y_400mf', 'regnet_y_800mf', 'regnet_y_8gf', 'resnet101', 'resnet152', 'resnet18', 'resnet34', 'resnet50', 'resnext101_32x8d', 'resnext101_64x4d', 'resnext50_32x4d', 'shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'shufflenet_v2_x1_5', 'shufflenet_v2_x2_0', 'squeezenet1

In [ ]:
model = models.__dict__['resnet50'](pretrained=True, num_classes=1000)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
activations = []
def save_activation(mod, inp, out):
    activations.append(out.view(out.shape[0], -1).detach().numpy())

name, module = list(model.named_modules())[-2]
print(f'Extracting output states from {name}:{module}')
module.register_forward_hook(save_activation)

Extracting output states from avgpool:AdaptiveAvgPool2d(output_size=(1, 1))


In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1v-KitwrisSYJU51b3vPMAKr1rVRbrirl' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1v-KitwrisSYJU51b3vPMAKr1rVRbrirl" -O toy_imagenet.zip && rm -rf /tmp/cookies.txt


--2022-11-16 17:58:47--  https://docs.google.com/uc?export=download&confirm=t&id=1v-KitwrisSYJU51b3vPMAKr1rVRbrirl
Resolving docs.google.com (docs.google.com)... 108.177.11.139, 108.177.11.101, 108.177.11.138, ...
Connecting to docs.google.com (docs.google.com)|108.177.11.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-08-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/n5jhnh29ihnra7k9k4850smd6lj2c37g/1668621525000/07900730216187465587/*/1v-KitwrisSYJU51b3vPMAKr1rVRbrirl?e=download&uuid=d1cbd3a0-5d97-4135-b79b-3d2475c4e708 [following]
--2022-11-16 17:58:48--  https://doc-08-08-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/n5jhnh29ihnra7k9k4850smd6lj2c37g/1668621525000/07900730216187465587/*/1v-KitwrisSYJU51b3vPMAKr1rVRbrirl?e=download&uuid=d1cbd3a0-5d97-4135-b79b-3d2475c4e708
Resolving doc-08-08-docs.googleusercontent.com (doc-08-08-docs.googleusercontent.com)... 173.194.214.132,

In [ ]:
!unzip toy_imagenet.zip


Archive:  toy_imagenet.zip
  inflating: toy_imagenet/great_white_shark/ILSVRC2012_val_00035774.JPEG  
  inflating: toy_imagenet/great_white_shark/ILSVRC2012_val_00024940.JPEG  
  inflating: toy_imagenet/great_white_shark/ILSVRC2012_val_00008848.JPEG  
  inflating: toy_imagenet/great_white_shark/ILSVRC2012_val_00037050.JPEG  
  inflating: toy_imagenet/great_white_shark/ILSVRC2012_val_00015220.JPEG  
  inflating: toy_imagenet/great_white_shark/ILSVRC2012_val_00004329.JPEG  
  inflating: toy_imagenet/great_white_shark/ILSVRC2012_val_00030127.JPEG  
  inflating: toy_imagenet/great_white_shark/ILSVRC2012_val_00014467.JPEG  
  inflating: toy_imagenet/great_white_shark/ILSVRC2012_val_00019409.JPEG  
  inflating: toy_imagenet/great_white_shark/ILSVRC2012_val_00039304.JPEG  
  inflating: toy_imagenet/great_white_shark/ILSVRC2012_val_00020011.JPEG  
  inflating: toy_imagenet/great_white_shark/ILSVRC2012_val_00034950.JPEG  
  inflating: toy_imagenet/great_white_shark/ILSVRC2012_val_00023185.JPEG 

In [ ]:
model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
  dataset = datasets.ImageFolder('toy_imagenet',
                                      transforms.Compose([transforms.Resize(int(224 * 1.14)),
                                                          transforms.CenterCrop(224),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                std=[0.229, 0.224, 0.225])]))
  loader = torch.utils.data.DataLoader(dataset, batch_size=50, shuffle=False, num_workers=8,
                                            pin_memory=True, sampler=SequentialSampler(dataset))
  for idx, (ims, targets) in enumerate(loader):
    print(idx*50)
    
    all_targets.append(targets)
    logits = model(ims)
    predictions = torch.argmax(logits, dim=-1).detach().numpy()
    all_preds.append(predictions)
predictions = np.concatenate(all_preds)
labels = torch.cat(all_targets).detach().numpy()
imvecs = np.concatenate(activations[-8:])
print(imvecs.shape)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


0
50
100
150
200
250
300
350
(400, 2048)


In [ ]:
print(predictions.shape)
print(labels.shape)
print(imvecs.shape)
print(len(activations))

(400,)
(400,)
(400, 2048)
16


In [ ]:
import pandas as pd
df = pd.DataFrame(columns=['label', 'prediction', 'cluster', 'path'])
df['prediction'] = predictions
df['label'] = labels
paths = [img[0] for img in dataset.imgs]
df['path'] = paths

In [ ]:
last_score = -2.
score = -1.
scaler = StandardScaler()
scaler.fit(imvecs)
X = scaler.transform(imvecs)
# #############################################################################
n_clusters = 2


# while score > last_score:
for i in range(2,10):
  print(f'Fitting to {n_clusters} clusters')
  last_score = score
  kmeans = KMeans(n_clusters=n_clusters, random_state=200)
  kmeans.fit(X)
  labels = kmeans.predict(X)
  score = metrics.silhouette_score(X, labels)
  if score > last_score: 
    best_model = kmeans
    best_labels = labels
  n_clusters += 1
  print(f'score: {score}')


Fitting to 2 clusters
score: 0.1028507798910141
Fitting to 3 clusters
score: 0.11581867188215256
Fitting to 4 clusters
score: 0.1505911946296692
Fitting to 5 clusters
score: 0.15570393204689026
Fitting to 6 clusters
score: 0.1568644940853119
Fitting to 7 clusters
score: 0.14705796539783478
Fitting to 8 clusters
score: 0.14899522066116333
Fitting to 9 clusters
score: 0.12387605756521225


In [ ]:
df['cluster'] = best_labels

In [ ]:

df

,label,prediction,cluster,path
0,0,981,5,toy_imagenet/ballplayer/ILSVRC2012_val_0000003...
1,0,489,5,toy_imagenet/ballplayer/ILSVRC2012_val_0000210...
2,0,981,5,toy_imagenet/ballplayer/ILSVRC2012_val_0000225...
3,0,429,5,toy_imagenet/ballplayer/ILSVRC2012_val_0000228...
4,0,981,5,toy_imagenet/ballplayer/ILSVRC2012_val_0000275...
...,...,...,...,...
395,7,970,2,toy_imagenet/volcano/ILSVRC2012_val_00044072.JPEG
396,7,980,2,toy_imagenet/volcano/ILSVRC2012_val_00045671.JPEG
397,7,980,2,toy_imagenet/volcano/ILSVRC2012_val_00046275.JPEG
398,7,980,2,toy_imagenet/volcano/ILSVRC2012_val_00046697.JPEG
